In [1]:
# !pip install interpret
# !pip install --user xgboost
# !pip install pytorch-tabnet
# !pip install anchor-exp

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from interpret import preserve, show
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

random_state=42

In [3]:
dataset_folder = 'dataset/ESDRPD/'

file = 'Dataset 2 _ Early-stage diabetes risk prediction dataset (ESDRPD).xlsx'
class_names = [0, 1]
_class = 'Class'

In [4]:
df = pd.read_excel(open(dataset_folder + file, 'rb'), sheet_name='Dataset 2 – Early-stage diabete')
 
df.replace(('Yes', 'No'), (1, 0), inplace=True)
df.replace(('Positive', 'Negative'), (1, 0), inplace=True)
df.replace(('Male', 'Female'), (1, 0), inplace=True)
df.head()

,Age,Gender,Polyuria,Polydipsia,Sudden weight loss,Weakness,Polyphagia,Genital thrush,Visual blurring,Itching,Irritability,Delayed healing,Partial paresis,Muscle stiffness,Alopecia,Obesity,Class
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1


In [5]:
X, y = df.drop(columns=[_class]), df[_class]

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=random_state)

In [7]:
x_test.shape

(156, 16)

# ML Model

In [8]:
import sklearn.metrics
import pandas as pd
import time
import numpy as np

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
random_state=42

In [11]:
x_train

,Age,Gender,Polyuria,Polydipsia,Sudden weight loss,Weakness,Polyphagia,Genital thrush,Visual blurring,Itching,Irritability,Delayed healing,Partial paresis,Muscle stiffness,Alopecia,Obesity
225,29,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
94,36,0,1,1,0,1,1,0,1,1,0,1,1,0,0,0
462,57,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0
284,72,1,1,0,0,0,1,0,1,1,0,1,1,1,1,0
23,48,1,0,1,1,1,0,0,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,35,0,0,1,1,1,0,0,0,1,0,1,1,1,0,0
106,58,1,0,1,1,1,1,0,1,1,0,0,1,0,1,1
270,40,0,1,1,1,1,0,0,1,0,0,1,1,1,0,0
435,57,1,1,1,1,1,1,0,1,0,0,0,1,0,0,0


In [12]:
def classify_report(clfs, dataset):
    x_train, y_train, x_test, y_test = dataset
    data = []
    for clf, name, no_df in clfs:
        if no_df:
            x_tr, x_te = x_train, x_test
        else:
            x_tr, x_te = x_train.values,  x_test.values
        clf.fit(x_tr, y_train)
        pred = clf.predict(x_te)
        f1, acc = sklearn.metrics.f1_score(y_test, pred, average='binary'), sklearn.metrics.accuracy_score(y_test, pred)
        data.append([name, f1, acc])
    df = pd.DataFrame(data, columns = ['Name', 'F1', 'Acc.'])
    df = df.sort_values(by=['F1'])
    return df

In [13]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=random_state)

# clfs = [(rf, 'rf', True), (gbc, 'gbc', True), (_xgb, 'xgb', True), (ebm, 'ebm', True), (tbn, 'tbn', False)]
# clfs = [(rf, 'rf', True), (ebm, 'ebm', True), (tbn, 'tbn', False)]
clfs = [(rf, 'rf', True)]
dataset = x_train, y_train, x_test, y_test
classify_report(clfs, dataset)

,Name,F1,Acc.
0,rf,0.995074,0.99359


# XAI

In [14]:
class_names = ['NO','YES']
feature_names = x_train.columns.to_list()
# local = lime, shap, anchor, tabnet, ebm
# global = pfi, tabnet, ebm, shap
# methods = pfi, tabnet, ebm, shap, lime, anchor
# remaining = pdp, eli5, ice, adawhip, break down

### Anchors (Rules)

In [15]:
from anchor import utils
from anchor import anchor_tabular
import re

In [16]:
exp_anchor = anchor_tabular.AnchorTabularExplainer(
    class_names,
    feature_names,
    x_train.values[:,:],
    {})

In [17]:
len(x_test), (len(x_test) * 3/3)/(60*60)

(156, 0.043333333333333335)

In [18]:
import time, datetime
from joblib import Parallel, delayed
import itertools

n_jobs = 8

def anchor_explain_instance_step(i, lim, threshold):
    warnings.filterwarnings("ignore")
    out = []
    for k in range(i, lim):
        e = exp_anchor.explain_instance(x_test.iloc[k, :].values, rf.predict, threshold=threshold)
        out.append(e)
    return out

def prepare_anchor_explanations():
    anchor_explanations_index = {'threshold': {}}
    total = len(x_test)
    # total = 100
    for threshold in [0.9, 0.95, 0.99]:
        now = datetime.datetime.now()
        print('threshold', threshold, 'at', now)
        start = time.time()
        # anchor_explanations_index['threshold'][threshold] = [exp_anchor.explain_instance(x_test.iloc[i, :].values, rf.predict, threshold=threshold) for i in range(total)]
        # out = Parallel(n_jobs=n_jobs)(delayed(anchor_explain_instance_step)(i, (i) + n_jobs, threshold) for i in range(0, total, n_jobs))
        out = Parallel(n_jobs=n_jobs)(delayed(anchor_explain_instance_step)(i, min(total, i+int(total/n_jobs)), threshold) for i in range(0, total, int(total/n_jobs)))
        anchor_explanations_index['threshold'][threshold] = list(itertools.chain(*out))
        del out
        taken = time.time() - start
        print(f'> done; taken {taken:.2f} at throughput {taken/total:.2f}')
    anchor_explanations_index['pred'] = [exp_anchor.class_names[rf.predict([x_test.iloc[i, :].values])[0]] for i in range(total)] # it is basically original model's prediction
    anchor_explanations_index['total'] = total
    return anchor_explanations_index

try: del anchor_explanations_index; 
except: pass
anchor_explanations_index = prepare_anchor_explanations()

threshold 0.9 at 2024-06-27 17:24:44.559963
> done; taken 39.54 at throughput 0.25
threshold 0.95 at 2024-06-27 17:25:24.102515
> done; taken 32.28 at throughput 0.21
threshold 0.99 at 2024-06-27 17:25:56.378955
> done; taken 36.57 at throughput 0.23


In [19]:
def get_anchor_metrics(threshold=1.0):
    out = []
    for i in range(anchor_explanations_index['total']):
        exp = anchor_explanations_index['threshold'][threshold][i]
        out.append([exp.precision(), exp.coverage(), len(exp.features())])
    precison, coverage, simplicity = np.array(out).mean(axis=0)
    print('Threshold at', threshold)
    print(f' Precison: {precison:.2f}')
    print(f' Coverage: {coverage:.2f}')
    print(f' Simplicity: {simplicity:.2f}')

for thres in anchor_explanations_index['threshold']:
    get_anchor_metrics(threshold=thres)

Threshold at 0.9
 Precison: 0.93
 Coverage: 0.36
 Simplicity: 2.15
Threshold at 0.95
 Precison: 0.99
 Coverage: 0.21
 Simplicity: 3.30
Threshold at 0.99
 Precison: 1.00
 Coverage: 0.19
 Simplicity: 3.66


In [20]:
rf_predict_array = rf.predict(x_test)

In [21]:
rf_predict_array[1]

1

In [22]:
from sklearn.metrics import accuracy_score

achor_predict_array = np.array([{'YES': 1, 'NO': 0}[anchor_explanations_index['pred'][i]] for i in range(len(x_test))])  # this is orignal prediction of the model

print(f'Anchors Fidelity (Predict): {accuracy_score(rf_predict_array, achor_predict_array):.4f}')
print('Not Applicable')

Anchors Fidelity (Predict): 1.0000
Not Applicable


In [23]:
def add_noise(data, noise_level=0.01):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

x_test_noisy = add_noise(x_test)
display(x_test_noisy)

,Age,Gender,Polyuria,Polydipsia,Sudden weight loss,Weakness,Polyphagia,Genital thrush,Visual blurring,Itching,Irritability,Delayed healing,Partial paresis,Muscle stiffness,Alopecia,Obesity
275,72.008268,1.006419,1.016963,-0.004484,0.005537,-0.000411,1.004047,-0.008901,1.005315,1.003791,0.015047,1.001912,0.993350,1.007431,0.993930,0.007517
93,39.999960,-0.009816,0.987821,0.997018,-0.006483,0.993467,0.998472,0.002321,-0.022711,1.017021,-0.002949,-0.004110,0.998625,0.000070,-0.002981,0.005206
6,56.996463,0.993331,0.986924,1.013709,0.005624,1.003709,0.985693,0.990743,0.001544,-0.003861,-0.013750,1.004374,1.005388,-0.017482,-0.002485,-0.004408
167,41.009635,1.012264,0.984939,1.004426,0.998529,0.984033,1.005618,1.005523,0.987245,0.994615,0.991931,0.013872,-0.000262,-0.008537,-0.006074,0.983445
90,44.988546,0.008968,-0.013578,0.004461,0.015648,-0.008150,1.031676,0.001953,0.990647,1.001621,-0.000177,0.005697,0.976790,-0.005130,-0.021400,-0.014341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,58.002480,1.013738,1.016368,0.997725,1.010055,0.996737,1.008708,-0.009918,0.996981,-0.005488,-0.006439,1.009768,1.001060,0.990853,-0.004002,0.991057
17,67.004026,0.985881,-0.000613,1.006796,-0.012226,1.004180,0.990745,0.017769,1.001332,0.000561,1.015449,0.999010,0.994728,1.001592,1.002349,0.988159
245,29.993140,1.021226,-0.020926,0.004451,-0.026157,-0.004440,-0.005241,-0.018654,0.017530,-0.000989,-0.013211,-0.002534,0.023745,0.005506,0.000068,0.010471
66,29.991902,0.003751,1.000042,0.990537,-0.005573,0.024750,-0.005667,-0.014853,0.000182,0.985704,-0.003913,0.998750,0.998496,-0.005402,0.002061,-0.000350


In [24]:
import time, datetime
from joblib import Parallel, delayed
import itertools

def anchor_explain_instance_step_noisy(i, lim, threshold):
    warnings.filterwarnings("ignore")
    out = []
    for k in range(i, lim):
        e = exp_anchor.explain_instance(x_test_noisy.iloc[k, :].values, rf.predict, threshold=threshold)
        out.append(e)
    return out

def prepare_anchor_explanations_noisy():
    anchor_explanations_index = {'threshold': {}}
    total = len(x_test_noisy)
    # total = 100
    for threshold in [0.9, 0.95, 0.99]:
        now = datetime.datetime.now()
        print('threshold', threshold, 'at', now)
        start = time.time()
        out = Parallel(n_jobs=n_jobs)(delayed(anchor_explain_instance_step_noisy)(i, min(total, i+int(total/n_jobs)), threshold) for i in range(0, total, int(total/n_jobs)))
        anchor_explanations_index['threshold'][threshold] = list(itertools.chain(*out))
        del out
        taken = time.time() - start
        print(f'> done; taken {taken:.2f} at throughput {taken/total:.2f}')
    anchor_explanations_index['total'] = total
    return anchor_explanations_index

try: del anchor_explanations_index_noisy; 
except: pass
anchor_explanations_index_noisy = prepare_anchor_explanations_noisy()

threshold 0.9 at 2024-06-27 17:26:35.682299
> done; taken 34.85 at throughput 0.22
threshold 0.95 at 2024-06-27 17:27:10.530717
> done; taken 40.07 at throughput 0.26
threshold 0.99 at 2024-06-27 17:27:50.598708
> done; taken 47.14 at throughput 0.30


In [25]:
def get_features_in_rules(exp_anchors):
    features = []
    for anchor_name in exp_anchors:
        idx = re.search('<|>|=', anchor_name).start()
        features.append(anchor_name[:idx-1])
    return features

def change_in_features(f1, f2):
    s1 = set(f1)
    s2 = set(f2)
    return 1- (len(s1.intersection(s2))/len(s1.union(s2)))


def calculate_robustness(explainer, total):
    for threshold in anchor_explanations_index['threshold']:
        robustness_list = []
        for i in range(total):
            exp = anchor_explanations_index['threshold'][threshold][i]
            noisy_exp = anchor_explanations_index_noisy['threshold'][threshold][i]
            delta = change_in_features(get_features_in_rules(exp.names()), get_features_in_rules(noisy_exp.names()))
            # print(delta)
            # print(get_features_in_rules(exp.names()), get_features_in_rules(noisy_exp.names()))
            robustness_list.append(delta)
        print('Threshold at', threshold, np.mean(robustness_list) if robustness_list else 0)
    return 

calculate_robustness(exp_anchor, len(x_test[:]))

Threshold at 0.9 0.3802218614718615
Threshold at 0.95 0.46616357289434207
Threshold at 0.99 0.4715381306727461


## Rule based explanations, alternate binary casese

In [26]:
ids_to_explain_list = [7, 8]

for _id in ids_to_explain_list:
    print('ID', _id,  'Prediction: ', exp_anchor.class_names[rf.predict([x_test.iloc[_id, :].values])[0]])
    np.random.seed(random_state)

    # Original explanation cases
    # Noisy explanation to compare how the rule (features in the rule) changes due to insertion of Noise
    for _indexes in [('Original', x_test), ('Noisy', x_test_noisy)]:
        msg, dataset = _indexes
        print('>', msg)
        exp = exp_anchor.explain_instance(dataset.iloc[_id, :].values, rf.predict, threshold=0.99)
        print('  >> Anchor: %s' % (' AND '.join(exp.names())))
        print('  >> Precision: %.2f' % exp.precision())
        print('  >> Coverage: %.2f' % exp.coverage())

ID 7 Prediction:  NO
> Original
  >> Anchor: Gender > 0.00 AND Polyuria <= 0.00 AND Polydipsia <= 0.00 AND Visual blurring <= 0.00 AND Partial paresis > 0.00
  >> Precision: 1.00
  >> Coverage: 0.02
> Noisy
  >> Anchor: Gender > 1.00 AND Polydipsia <= 0.00 AND Polyphagia <= 0.00 AND Alopecia <= 0.00 AND Genital thrush <= 0.00 AND Obesity > 0.00 AND Partial paresis <= 1.00 AND Age <= 48.00
  >> Precision: 1.00
  >> Coverage: 0.00
ID 8 Prediction:  YES
> Original
  >> Anchor: Polydipsia > 0.00 AND Muscle stiffness <= 0.00 AND Obesity <= 0.00 AND Weakness <= 0.00
  >> Precision: 1.00
  >> Coverage: 0.05
> Noisy
  >> Anchor: Polydipsia > 0.00 AND Muscle stiffness <= 0.00 AND Partial paresis <= 0.00 AND Irritability > 0.00
  >> Precision: 1.00
  >> Coverage: 0.01
